<a href="https://colab.research.google.com/github/Sidhtang/muisc-reccomendation-systewm-using-transformer/blob/main/music_recommendation_system_using_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from datasets import load_dataset ,Audio
import numpy as np
from transformers import pipeline,AutoFeatureExtractor,AutoModelForAudioClassification,TrainingArguments,Trainer

In [ ]:
gtzan= load_dataset("marsyas/gtzan","all")
gtzan=gtzan["train"].train_test_split(seed=42,shuffle=True,test_size=0.1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for marsyas/gtzan contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/marsyas/gtzan.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)
sampling_rate = feature_extractor.sampling_rate
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))
sample = gtzan["train"][0]["audio"]
inputs = feature_extractor(
    sample["array"], sampling_rate=sample["sampling_rate"])
max_duration = 20.0
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs


gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=25,
    num_proc=1,
)

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")
id2label_fn = gtzan["train"].features["genre"].int2str
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

model_name = model_id.split("/")[-1]
batch_size = 2
gradient_accumulation_steps = 1
num_train_epochs = 5

training_args = TrainingArguments(
    f"{model_name}-Music classification Finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
)

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.503000,1.294158,0.660000
2,1.422700,0.955839,0.700000
3,0.107700,0.604249,0.820000
4,0.429000,0.941155,0.770000
5,0.099700,0.732464,0.810000


TrainOutput(global_step=2250, training_loss=0.847630963643392, metrics={'train_runtime': 1949.1771, 'train_samples_per_second': 2.306, 'train_steps_per_second': 1.154, 'total_flos': 2.044662758208e+17, 'train_loss': 0.847630963643392, 'epoch': 5.0})

In [ ]:
# Save the model and feature extractor
model.save_pretrained("/content/Saved Model")
feature_extractor.save_pretrained("/content/Saved Model")

['/content/Saved Model/preprocessor_config.json']

In [ ]:
from transformers import pipeline, AutoFeatureExtractor, AutoModelForAudioClassification

# Load the saved model and feature extractor
model_path = "/content/Saved Model"  # Update with your actual path
loaded_model = AutoModelForAudioClassification.from_pretrained(model_path)
loaded_feature_extractor = AutoFeatureExtractor.from_pretrained(model_path)

pipe = pipeline("audio-classification", model=loaded_model,
                feature_extractor=loaded_feature_extractor)

def classify_audio(audio_path="/content/sb_indreams(chosic.com).mp3"):
    preds = pipe(audio_path)
    outputs = {}
    for p in preds:
        outputs[p["label"]] = p["score"]
    return outputs


# Provide the input file path
input_file_path = input('/content/sb_indreams(chosic.com).mp3')  # You might want to remove this line as it doesn't seem to be used

# Classify the audio file
output = classify_audio("/content/sb_indreams(chosic.com).mp3")

# Print the output genre
print(output)
print("Predicted Genre:")
max_key = max(output, key=output.get)

print("The predicted genre is:", max_key)
print("The prediction score is:", output[max_key])

/content/sb_indreams(chosic.com).mp3k
{'country': 0.3373990058898926, 'rock': 0.2830536365509033, 'classical': 0.251537948846817, 'metal': 0.04294976964592934, 'blues': 0.029479769989848137}
Predicted Genre:
The predicted genre is: country
The prediction score is: 0.3373990058898926
